In [1]:
import pandas as pd
import os
import helper as analytics
import time
from dotenv import load_dotenv
from helper import calculate_scores, get_epoch_from_checkpoint
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [15]:
files = analytics.get_all_files_in_directory("../../results/gpt-4o-mini/wdc-small-with-synthetic", ".jsonl")

for file in files:
    df = pd.read_json(file, lines=True)
    starting_length = len(df)
    # drop record if the custom id includes the words "dblp-acm" "dblp-scholar"
    df = df[~df["custom_id"].str.contains("dblp-acm")]
    df = df[~df["custom_id"].str.contains("dblp-scholar")]
    ending_length = len(df)
    print(f"Removed {starting_length - ending_length} records from {file}")
    # save the file
    df.to_json(f"{file}", orient='records', lines=True)
    

Removed 32860 records from ../../results/gpt-4o-mini/wdc-small-with-synthetic/batch_TK8yBid20mYnB7ziRsYknVg8_output.jsonl
Removed 0 records from ../../results/gpt-4o-mini/wdc-small-with-synthetic/batch_7ymSj1XZ6YlwZe4b5amY02v3_output.jsonl


In [2]:
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    return accuracy, f1, precision, recall

In [3]:
def list_checkpoint_folders(directory):
    checkpoint_folders = []
    
    for root, dirs, files in os.walk(directory):
        for folder in dirs:
            if 'checkpoint' in folder:
                checkpoint_folders.append(os.path.join(root, folder))
                
    return checkpoint_folders

# Function to extract the checkpoint number
def get_checkpoint_number(path):
    return int(path.split('-')[-1])

In [4]:
included_tasks=["domain-complex-free (Product)", "domain-simple-free (Product)", "domain-complex-force (Product)", "domain-simple-force (Product)"]

In [5]:
RESULT_DIR = "../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/abt-buy/no_explanations/2024-08-14-15-50-23/results"

In [6]:
experiment_paths = analytics.get_all_files_in_directory(RESULT_DIR)

stats_dataframes = []

for experiment_path in experiment_paths:
    # Load the dataset
    dataset_name = experiment_path.split("/")[-2]
    print(f"Processing {dataset_name}")
    df = pd.read_json(experiment_path)
    # Calculate stats for the filtered DataFrame
    stats_df = analytics.calculate_stats(df)
    stats_df['Dataset'] = dataset_name  # Add dataset name for reference
    stats_dataframes.append(stats_df)
    
result_df = pd.concat(stats_dataframes)
result_df.to_csv(f"{RESULT_DIR}/stats.csv", index=False)

Processing amazon-google-full
Processing wdc-fullsize
Processing abt-buy-full
Processing walmart-amazon
Processing dblp-scholar
Processing dblp-acm


In [7]:
result_df

,Task,Accuracy,F1 Score,Precision,Recall,Num -1 Responses,Dataset
0,domain-complex-free (Product),0.898823,0.567164,0.503311,0.649573,0,amazon-google-full
1,domain-simple-free (Product),0.887484,0.565657,0.466667,0.717949,0,amazon-google-full
2,domain-complex-force (Product),0.897078,0.561338,0.496711,0.645299,0,amazon-google-full
3,domain-simple-force (Product),0.894025,0.591597,0.487535,0.752137,0,amazon-google-full
0,domain-complex-free (Product),0.930889,0.627545,0.782090,0.524000,0,wdc-fullsize
1,domain-simple-free (Product),0.931556,0.653933,0.746154,0.582000,0,wdc-fullsize
2,domain-complex-force (Product),0.928222,0.631699,0.734748,0.554000,0,wdc-fullsize
3,domain-simple-force (Product),0.932222,0.660734,0.744361,0.594000,0,wdc-fullsize
0,domain-complex-free (Product),0.971294,0.850949,0.963190,0.762136,0,abt-buy-full
1,domain-simple-free (Product),0.972338,0.858667,0.952663,0.781553,0,abt-buy-full


In [5]:
result_df.groupby('Dataset')["F1 Score"].mean()

Dataset
wdc-fullsize    0.720788
Name: F1 Score, dtype: float64

## Continues training

In [34]:
def analyse_validation(checkpoint_path, new_model_name):
    result_file = f"{checkpoint_path}/validation_results.csv"
    checkpoint_paths = list_checkpoint_folders(checkpoint_path)
    # Sorting the list by the checkpoint number
    checkpoint_paths = sorted(checkpoint_paths, key=get_checkpoint_number)
    
    results = []
    # get all validation files
    for checkpoint_path in checkpoint_paths:
        if not os.path.exists(f"{checkpoint_path}/validation_results.json"):
            print(f"Validation results already exist for {checkpoint_path}")
            continue
        df = pd.read_json(f"{checkpoint_path}/validation_results.json")
        df.loc[df['chatbot_response_clean'] == -1, 'chatbot_response_clean'] = 0
        f1, precision, recall = calculate_scores(df)
        epoch = get_epoch_from_checkpoint(checkpoint_paths, int(checkpoint_path.split("/")[-1].replace("checkpoint-", "")))
        result = {
            "checkpoint_path": checkpoint_path,
            "checkpoint_number": checkpoint_path.split("/")[-1].replace("checkpoint-", ""),
            "epoch": epoch,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }
        results.append(result)
    df = pd.DataFrame(results)

    # sort by highest f1
    df_sorted = df.sort_values(by='f1', ascending=False)
    #df_sorted.to_csv("{checkpoint_path}/validation_results.csv", index=False)

    # get the checkpoint path for the best f1
    best_checkpoint_path = df_sorted.iloc[0]['checkpoint_path']
    print(f"Best Checkpoint Path: {best_checkpoint_path}")
    print(f"Best F1: {df_sorted.iloc[0]['f1']}")
    df_sorted.to_csv(result_file, index=False)
    return best_checkpoint_path

In [35]:
for i in range(1,6):
    checkpoint_path = f"../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-11_explanation/Meta-Llama-3.1-8B-Instruct-error-small_explanation-enhanced-{i}"
    new_model_name = f"Meta-Llama-3.1-70B-Instruct-error-small_explanation-enhanced-{i}"
    best_checkpoint_path = analyse_validation(checkpoint_path, new_model_name)
    print(f"Best Checkpoint Path: {best_checkpoint_path}")

Best Checkpoint Path: ../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-11_explanation/Meta-Llama-3.1-8B-Instruct-error-small_explanation-enhanced-1/checkpoint-286
Best F1: 0.7471852610030706
Best Checkpoint Path: ../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-11_explanation/Meta-Llama-3.1-8B-Instruct-error-small_explanation-enhanced-1/checkpoint-286
Best Checkpoint Path: ../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-11_explanation/Meta-Llama-3.1-8B-Instruct-error-small_explanation-enhanced-2/checkpoint-143
Best F1: 0.7523105360443623
Best Checkpoint Path: ../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-11_explanation/Meta-Llama-3.1-8B-Instruct-error-small_explanation-enhanced-2/checkpoint-143
Best Checkpoint Path: ../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/synthetic_textual/2024-08-12-14-59-1

## Batch job

In [7]:
def parse_response(response):
    body = response.get("body", {})
    usage = body.get("usage", {})
    choices = body.get("choices", [{}])
    message = choices[0].get("message", {}) if choices else {}

    return pd.Series({
        "status_code": response.get("status_code"),
        "request_id": response.get("request_id"),
        "completion_id": body.get("id"),
        "created": body.get("created"),
        "model": body.get("model"),
        "content": message.get("content"),
        "prompt_tokens": usage.get("prompt_tokens"),
        "completion_tokens": usage.get("completion_tokens"),
        "total_tokens": usage.get("total_tokens"),
    })

In [35]:
df = pd.read_json("../../results/gpt-4o-mini/tobedetermined/batch_s1Z0ng7isxQqHWGDdTQI70bM_output.jsonl", lines=True)["response"].apply(parse_response)

# calculate the average completion tokens

df["model"].iloc[0]

'ft:gpt-4o-mini-2024-07-18:wbsg-uni-mannheim::9z0GiK5t'

In [12]:
gpt_result_1_path = "../../results/gpt-4o-mini/wdc-small-synthetic-filtered-interesting/batch_EHm0OsyYK3mO3l61WDzDwRs6_output.jsonl"

gpt_result_1 = pd.read_json(gpt_result_1_path, lines=True)
gpt_result_2 = pd.read_json("../../results/gpt-4o-mini/wdc-small-synthetic-filtered-interesting/batch_y1qVVew2QgFfM1qzXuSOo0zD_output.jsonl", lines=True)

# concatenate the two dataframes
gpt_result = pd.concat([gpt_result_1, gpt_result_2])
# reset the index
gpt_result = gpt_result.reset_index(drop=True)

# split the custom_id into dataset, task and index
gpt_result[['dataset', 'task', 'pair_id', 'label']] = gpt_result.custom_id.str.split(";", expand=True)
gpt_result = gpt_result.drop(columns=['custom_id'])

# Apply the function to the response column
parsed_df = gpt_result["response"].apply(parse_response)

# Concatenate the parsed results with the original dataframe
gpt_result = pd.concat([gpt_result, parsed_df], axis=1)

In [13]:
# transform content to 0 or 1
gpt_result['content'] = gpt_result['content'].apply(lambda x: 1 if "Yes" in x else 0)

# change label from string to int
gpt_result['label'] = gpt_result['label'].astype(int)

# Assuming 'label' is the true label and 'response' is the predicted label
results = []

grouped = gpt_result.groupby(['dataset', 'task'])

for (dataset, task), group in grouped:
    y_true = group['label']
    y_pred = group['content']
    
    accuracy, f1, precision, recall = calculate_metrics(y_true, y_pred)
    
    results.append({
        'dataset': dataset,
        'task': task,
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    })

# Convert the results into a DataFrame
metrics_df = pd.DataFrame(results)
metrics_df.to_csv(f"{gpt_result_1_path.rsplit('/', 1)[0]}/stats.csv", index=False)

In [14]:
metrics_df

,dataset,task,accuracy,f1_score,precision,recall
0,abt-buy-full,domain-complex-force (Product),0.961900,0.843011,0.756757,0.951456
1,abt-buy-full,domain-complex-free (Product),0.959290,0.834746,0.740602,0.956311
2,abt-buy-full,domain-simple-force (Product),0.962944,0.844639,0.768924,0.936893
3,abt-buy-full,domain-simple-free (Product),0.962944,0.845316,0.766798,0.941748
4,amazon-google-full,domain-complex-force (Product),0.873528,0.594972,0.441909,0.910256
5,amazon-google-full,domain-complex-free (Product),0.873092,0.596394,0.441478,0.918803
6,amazon-google-full,domain-simple-force (Product),0.873092,0.593007,0.440748,0.905983
7,amazon-google-full,domain-simple-free (Product),0.865678,0.580381,0.426000,0.910256
8,dblp-acm,domain-complex-force (Product),0.854832,0.711647,0.553059,0.997748
9,dblp-acm,domain-complex-free (Product),0.852810,0.708333,0.549751,0.995495
